In [ ]:
!pip install outscraper pandas

In [ ]:
import pandas as pd
from outscraper import ApiClient
from datetime import date, timedelta, datetime
import time

# --- Configurações Iniciais ---
API_KEY = "" # API OUTRSCRAPPER

locais_para_pesquisar = [
    "" # URL DO ESTABELECIMENTO
]

# Definimos o número de avaliações a buscar
numero_de_avaliacoes = 500
idioma_avaliacoes = "pt-BR"
regiao_pesquisa = "BR"
ordenacao = "newest"

hoje = date.today()
dias_para_subtrair = hoje.weekday() + 1
ultimo_domingo = hoje - timedelta(days=dias_para_subtrair)
primeira_segunda = ultimo_domingo - timedelta(days=6)
data_inicio = datetime.combine(primeira_segunda, datetime.min.time())
data_fim = datetime.combine(ultimo_domingo, datetime.max.time())

print(f"Buscando avaliações da semana anterior:")
print(f"Período: {data_inicio.strftime('%d/%m/%Y %H:%M')} até {data_fim.strftime('%d/%m/%Y %H:%M')}\n")

client = ApiClient(api_key=API_KEY)
todas_as_avaliacoes = []

print("Iniciando a extração de avaliações...")

for local_query in locais_para_pesquisar:
    try:
        nome_display = local_query.split('q=')[1].split('&')[0].replace('+', ' ').replace('%27', "'").title()
    except IndexError:
        nome_display = local_query
    print(f"\nBuscando avaliações para: {nome_display}...")

    try:
        response = client.google_maps_reviews(
            local_query,
            reviews_limit=numero_de_avaliacoes,
            language=idioma_avaliacoes,
            region=regiao_pesquisa,
            sort=ordenacao
        )

        if response and isinstance(response, list) and len(response) > 0:
            business_data = response[0]
            business_name = business_data.get('name', 'Nome não encontrado')
            reviews_data = business_data.get('reviews_data', [])

            print(f"Encontradas {len(reviews_data)} avaliações recentes para '{business_name}'.")

            if reviews_data:
                for review in reviews_data:
                    review['business_name'] = business_name
                todas_as_avaliacoes.extend(reviews_data)
        else:
            print(f"Nenhum local ou avaliação encontrada para a URL.")

    except Exception as e:
        print(f"Ocorreu um erro ao buscar avaliações: {e}")

print("\nExtração finalizada.")

# --- Processamento, Filtragem e Exportação ---
if todas_as_avaliacoes:
    df = pd.DataFrame(todas_as_avaliacoes)
    df['review_datetime_utc'] = pd.to_datetime(df['review_datetime_utc'])
    df_filtrado = df[
        (df['review_datetime_utc'] >= data_inicio) &
        (df['review_datetime_utc'] <= data_fim)
    ].copy()
    print(f"\nTotal de {len(df_filtrado)} avaliações encontradas no período desejado.")
    if not df_filtrado.empty:
        colunas_desejadas = [
            'business_name', 'author_title', 'review_rating', 'review_text', 'review_datetime_utc'
        ]
        df_final = df_filtrado[colunas_desejadas].rename(columns={'business_name': 'name'})
        df_final.drop_duplicates(subset=['review_text', 'author_title', 'review_datetime_utc'], inplace=True)
        nome_do_arquivo = f"avaliacoes_{primeira_segunda.strftime('%Y-%m-%d')}_a_{ultimo_domingo.strftime('%Y-%m-%d')}.csv"
        df_final.to_csv(nome_do_arquivo, index=False, encoding='utf-8-sig')
        print("\nAmostra dos dados filtrados:")
        print(df_final.head())
        print(f"\nRelatório salvo com sucesso no arquivo: '{nome_do_arquivo}'!")
    else:
        print("\nNenhuma avaliação encontrada no período da semana anterior.")
else:
    print("\nNenhuma avaliação foi coletada.")